In [1]:
import pandas as pd
from sqlite_utils import Database

In [2]:
db = Database("mybinder_archive.db")

In [3]:
# get table names
db.table_names()

['mybinderlaunch', 'repo']

In [4]:
launch_table = 'mybinderlaunch'

### Total number of launches

In [5]:
# db[launch_table].columns
db[launch_table].columns_dict

{'timestamp': float,
 'version': int,
 'origin': str,
 'provider': str,
 'spec': str,
 'org': str,
 'ref': str,
 'resolved_ref': str,
 'repo_url': str}

In [6]:
db[launch_table].count

9624058

Timestamp of first launch

In [7]:
launches = db[launch_table].rows_where(order_by="timestamp")
first_launch_timestamp = next(launches)["timestamp"]
del launches

first_launch_timestamp

'2018-11-03 00:00:00+00:00'

Timestamp of last launch

In [8]:
launches = db[launch_table].rows_where(order_by="timestamp desc")
last_launch_timestamp = next(launches)["timestamp"]
del launches

last_launch_timestamp

'2020-06-30 23:59:00+00:00'

### Launches per origin

In [9]:
query = f'select origin, count(origin) from {launch_table} group by "origin";'
launches_per_origin = pd.read_sql_query(query, db.conn)
launches_per_origin

,origin,count(origin)
0,binder.mybinder.ovh,556
1,binder.mybinder.turing.ac.uk,129
2,gesis.mybinder.org,681920
3,gke.mybinder.org,4754321
4,gke.mybinder.org:443,1
5,mybinder.org,2741924
6,notebooks.gesis.org,668369
7,ovh.mybinder.org,535332
8,turing.mybinder.org,241506


In [10]:
launches_per_origin_dict = {"gke": 0, "gesis": 0, "turing": 0, "ovh": 0}
for index, row in launches_per_origin.iterrows():
    if row["origin"] == "mybinder.org":
        launches_per_origin_dict["gke"] += row["count(origin)"]
    for origin in launches_per_origin_dict.keys():
        if origin in row["origin"]:
            # print(origin, row["origin"])
            launches_per_origin_dict[origin] += row["count(origin)"]

assert sum(launches_per_origin_dict.values()) == db[launch_table].count

pd.DataFrame.from_dict(
    launches_per_origin_dict, 
    orient="index", 
    columns=["launches"]
).sort_values(by=["launches"], ascending=False)

,launches
gke,7496246
gesis,1350289
ovh,535888
turing,241635


### Launches per provider

In [11]:
query = f'select provider, count(provider) from {launch_table} group by "provider";'
launches_per_provider = pd.read_sql_query(query, db.conn).sort_values(by=["count(provider)"], ascending=False)

launches_per_provider

,provider,count(provider)
4,GitHub,9388681
2,Gist,131204
3,Git,63858
5,GitLab,38556
7,Zenodo,943
6,Hydroshare,383
1,Figshare,306
0,Dataverse,127


### Organisations/Users in repos

In [12]:
# number of different orgs
query = "select count(distinct org) from mybinderlaunch;"
db.conn.execute(query).fetchone()[0]

33514